In [0]:
# Define your config
storage_account_name = "adfprojectstorage"
container_name = "bronze"
relative_path = "Report (17).csv"
sas_token = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-05-31T13:27:04Z&st=2025-04-30T05:27:04Z&spr=https&sig=ju1VNkjB6Wu2JBGwu4L5T7t5MnhJjP0fmH6zvi7EfxY%3D"

# Set the Spark config to use the SAS token
spark.conf.set(f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net", sas_token)

# Use wasbs:// (not abfss://) for SAS
wasbs_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{relative_path}"

# Read the CSV file
raw_df = spark.read.format("csv").option("header", "true").load(wasbs_path)
display(raw_df)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, to_timestamp, expr, lpad,date_format

select_specfic_col_df = raw_df.select(col("Fault Number"),col("short Description"),col("Assignee Group"),col("submitter"),col("Create Date"),col("Service Outage Start Date/Time"))
display(select_specfic_col_df)

In [0]:

# Step 1: Convert string columns to timestamp
select_specfic_col_df = select_specfic_col_df.withColumn(
    "Create_Date", to_timestamp(col("Create Date"), "dd/MM/yyyy HH:mm")
).withColumn(
    "Service_Outage_Start", to_timestamp(col("Service Outage Start Date/Time"), "dd/MM/yyyy HH:mm")
)

select_specfic_col_df = select_specfic_col_df.withColumn(
    "Create_Date_dd/MM/yyyy", date_format(col("Create_Date"), "dd/MM/yyyy HH:mm")
).withColumn(
    "Service_Outage_Start_dd/MM/yyyy", date_format(col("Service_Outage_Start"), "dd/MM/yyyy HH:mm")
)

# Step 2: Calculate difference in seconds
select_specfic_col_df = select_specfic_col_df.withColumn(
    "alarm_seconds",
    unix_timestamp(col("Create_Date")) - unix_timestamp(col("Service_Outage_Start"))
)

# Step 3: Format seconds into HH:mm:ss
select_specfic_col_df = select_specfic_col_df.withColumn(
    "MTTD",
    expr("""
        concat(
            lpad(floor(alarm_seconds / 3600), 2, '0'), ':',
            lpad(floor((alarm_seconds % 3600) / 60), 2, '0'), ':',
            lpad(alarm_seconds % 60, 2, '0')
        )'
    """)
)

# Step 4: Display the result
display(select_specfic_col_df.select("Fault Number","short Description","Assignee Group","submitter","Create Date", "Create_Date_dd/MM/yyyy",'Create_Date',"Service Outage Start Date/Time", "Service_Outage_Start_dd/MM/yyyy",'Service_Outage_Start',"alarm_seconds","MTTD"))


In [0]:
final_df = select_specfic_col_df.select("Fault Number","short Description","Assignee Group","submitter", "Create_Date_dd/MM/yyyy", "Service_Outage_Start_dd/MM/yyyy","MTTD")
display(final_df)